In [ ]:
import numpy as np
import pandas as pd
import time
import requests
import re
from datetime import datetime
from datetime import timedelta

import json
from ip2geotools.databases.noncommercial import DbIpCity

In [ ]:
path = '../Data/Ips_export.csv'

Ips_raw = pd.read_csv(path, engine='python')

Ips_raw.head()

In [ ]:
Ips_raw.tail()

In [ ]:
Ips_copy = Ips_raw.copy()
ips = Ips_copy.IP.unique()

In [ ]:
# Function for extract Locations

localhost = "127.0.0.1"

def extract_location(ip):
    
    try:
        if ip == localhost:
            return ('Local', 'Local')

        else:
            response = DbIpCity.get(ip, api_key = 'free').to_json()    
            data = json.loads(response)
            return data['city'], data['country']
    
    except:
        return np.nan

In [ ]:
# Extract Location Test

extract_location(Ips_copy.IP[261868])

In [ ]:
# List of locations for unique Ips
start = time.monotonic()

locations = []

for item in ips:
    
    locations.append(extract_location(item))  

end = time.monotonic()

print(timedelta(seconds = end - start))

In [ ]:
# Compare ips & add location
start = time.monotonic()

IP_locations = []
count = 1

for index, row in Ips_copy.iterrows():
    
    # iterate the two lists
    for a, b in zip(ips, locations):
        
        # Compare & add
        if row.IP == a:
            IP_locations.append(b)  
            print(b, count)
            count += 1

end = time.monotonic()

print(timedelta(seconds = end - start))

In [ ]:
Ips_copy["Location"] = IP_locations

In [ ]:
Ips_copy

In [ ]:
Locations_export= Ips_copy.copy()

Locations_export.replace('', 'null', inplace = True)

Locations_export.to_csv('../Data/Locations_export.csv', index = False)